In [1]:
%pip install mediapipe
%pip install tensorflow
# Add other necessary packages
%%python -m pip install mediapipe


                                              0.0/50.5 MB ? eta -:--:--
                                             0.0/50.5 MB 991.0 kB/s eta 0:00:51
                                              0.4/50.5 MB 4.6 MB/s eta 0:00:11
                                              0.8/50.5 MB 6.0 MB/s eta 0:00:09
                                              1.2/50.5 MB 7.1 MB/s eta 0:00:07
     -                                        1.8/50.5 MB 8.2 MB/s eta 0:00:06
     -                                        2.4/50.5 MB 9.0 MB/s eta 0:00:06
     --                                       3.0/50.5 MB 9.7 MB/s eta 0:00:05
     --                                       3.8/50.5 MB 10.4 MB/s eta 0:00:05
     ---                                      4.4/50.5 MB 10.9 MB/s eta 0:00:05
     ---                                      5.0/50.5 MB 11.1 MB/s eta 0:00:05
     ----                                     5.5/50.5 MB 11.0 MB/s eta 0:00:05
     ----                                     6.2/50.5


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: C:\Users\coldf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


                                              0.0/300.9 MB ? eta -:--:--
                                              0.1/300.9 MB 5.5 MB/s eta 0:00:56
                                             1.0/300.9 MB 12.5 MB/s eta 0:00:25
                                             2.0/300.9 MB 16.0 MB/s eta 0:00:19
                                             2.9/300.9 MB 18.4 MB/s eta 0:00:17
                                             3.1/300.9 MB 16.8 MB/s eta 0:00:18
                                             3.8/300.9 MB 15.2 MB/s eta 0:00:20
                                             4.8/300.9 MB 15.2 MB/s eta 0:00:20
                                             5.8/300.9 MB 17.0 MB/s eta 0:00:18
                                             6.8/300.9 MB 17.5 MB/s eta 0:00:17
     -                                       7.9/300.9 MB 17.9 MB/s eta 0:00:17
     -                                       8.1/300.9 MB 16.2 MB/s eta 0:00:19
     -                                       9.

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\coldf\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: C:\Users\coldf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [2]:
import mediapipe as mp


In [3]:
# import mediapipe as mp
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras import layers, models

import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np

from mediapipe.tasks import python
from mediapipe.tasks.python import audio

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Function to perform facial landmarking on an image
def perform_facial_landmarking(image_path):
    # Read image
    image = cv2.imread(image_path)

    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image
    results = face_mesh.process(rgb_image)

    # Extract landmarks
    landmarks = []
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                landmarks.append([x, y, z])

    return np.array(landmarks)

# Folder containing FER2013 dataset images
dataset_folder = 'dataset/train'  # Adjust the folder path accordingly

# Apply facial landmarking to all images in subfolders
facial_landmarks_data = []

for emotion_folder in os.listdir(dataset_folder):
    emotion_path = os.path.join(dataset_folder, emotion_folder)
    print('Processing emotion:', emotion_folder)
    if os.path.isdir(emotion_path):
        for image_file in os.listdir(emotion_path):
            image_path = os.path.join(emotion_path, image_file)
            landmarks = perform_facial_landmarking(image_path)
            facial_landmarks_data.append({'landmarks': landmarks.tolist(), 'emotion': emotion_folder, 'image_path': image_path})

# Convert the result into a DataFrame
landmarks_df = pd.DataFrame(facial_landmarks_data)

# Save the DataFrame with facial landmarks, emotions, and image paths to a CSV file
landmarks_df.to_csv('fer2013_landmarks.csv', index=False)

# Training the Model

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('landmarked_dataset.csv')

In [ ]:
# Pre-process the data
X = df.drop('emotion_label', axis=1).values
y = df['emotion_label'].values

In [ ]:
# Encode Labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Define the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Make predictions
predictions = model.predict(X_test)
